## Writing catalogs to disk

There's a new method to save catalogs to disk:

```python
class Catalog:
    ...

    def write_catalog(
        self,
        base_catalog_path: str | Path | UPath,
        *,
        catalog_name: str | None = None,
        default_columns: list[str] | None = None,
        as_collection: bool = True,
        overwrite: bool = False,
        **kwargs, # kwargs for `NestedFrame.to_parquet`
    )
```

We can write:
- Collections (main catalog, margin, and appropriate properties).
- Standalone catalogs, providing `as_collection=False`.

In [1]:
import lsdb

small_sky_collection = lsdb.open_catalog("../tests/data/small_sky_order1_collection")
small_sky_collection

,id,ra,dec,ra_error,dec_error
npartitions=4,,,,,
"Order: 1, Pixel: 44",int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],int64[pyarrow]
"Order: 1, Pixel: 45",...,...,...,...,...
"Order: 1, Pixel: 46",...,...,...,...,...
"Order: 1, Pixel: 47",...,...,...,...,...


By default a catalog is saved as a collection:

In [2]:
small_sky_collection.write_catalog(
    "my_small_sky_collection",
    catalog_name="my_small_sky",
    default_columns=["id","ra","dec"],
)

In [3]:
%ls my_small_sky_collection/

collection.properties  my_small_sky/          my_small_sky_3600arcs/


Writing the main catalog only:

In [4]:
small_sky_collection.write_catalog(
    "my_small_sky",
    catalog_name="my_small_sky",
    default_columns=["id","ra","dec"],
    as_collection=False, # added this flag
)

In [5]:
%ls my_small_sky/

dataset/            partition_info.csv  properties
hats.properties     point_map.fits      skymap.fits


### Updates to HATS creation properties

These properties, often carried over from hats-import, were not being updated after processing with LSDB.

```plain
hats_builder=lsdb v0.6.2.dev26+ga9279dbe.d20250721, hats v0.6.2.dev3+ge7fdab4e
hats_creation_date=2025-07-24T13\:42UTC
hats_estsize=33
```

In [6]:
%cat my_small_sky_collection/my_small_sky/hats.properties

#HATS catalog
obs_collection=my_small_sky
dataproduct_type=object
hats_nrows=131
hats_col_ra=ra
hats_col_dec=dec
hats_cols_default=id ra dec
hats_npix_suffix=.parquet
hats_skymap_order=1
obs_regime=Optical
default_index=id
hats_builder=lsdb v0.6.2.dev26+ga9279dbe.d20250721, hats v0.6.2.dev3+ge7fdab4e
hats_creation_date=2025-07-24T14\:17UTC
hats_estsize=33
hats_release_date=2024-09-18
hats_version=v0.1
hats_max_rows=42
hats_order=1
moc_sky_fraction=0.08333


In [7]:
!diff \
../tests/data/small_sky_order1_collection/small_sky_order1/hats.properties \
my_small_sky_collection/my_small_sky/hats.properties

2c2
< obs_collection=small_sky_order1
---
> obs_collection=my_small_sky
6a7
> hats_cols_default=id ra dec
11,12c12,13
< hats_builder=hats-import v0.6.1.dev1+gd082cb2, hats v0.7.dev0+g8d490830.d20250613
< hats_creation_date=2025-06-16T18\:06UTC
---
> hats_builder=lsdb v0.6.2.dev26+ga9279dbe.d20250721, hats v0.6.2.dev3+ge7fdab4e
> hats_creation_date=2025-07-24T14\:17UTC
16c17
< hats_max_rows=1000000
---
> hats_max_rows=42
